In [ ]:
import torch
import pandas as pd 
from ultralytics import YOLO
import numpy as np
import supervision as sv
import logging
from tkinter import Tk,filedialog
import os 
from pathlib import Path
import csv
import cv2

In [ ]:
model = YOLO('../Models/2025_v0.pt')
#source_file =  add code for file diaglouge to open to laod video 
#target_file = get the file name freom sourcepath and add_processed before the file extgension. 
logging.basicConfig(level=logging.INFO)
logging.info('Started Inference')

root=Tk()
root.withdraw()
source_file = filedialog.askopenfilename(title="Select source video file",filetypes=[("Video files", "*.mp4;*.avi;*.mov"), ("All files", "*.*")])
if not source_file:
    logging.error("No source file selected. Exiting.")
    exit()
directory,filename = os.path.split(source_file)
name,ext = os.path.splitext(filename)
target_file_name = name + '_processed' + ext
target_file = os.path.join(directory,target_file_name)

In [ ]:
def run_inference(
    model,
    frame,
    frame_number,
    save_csv: bool = False,
    conf_threshold: float = 0.25,
    writer=None
):
    """
    Run YOLO → annotate → optionally log to CSV.

    Args:
      model           : YOLO model instance
      frame           : HxWx3 numpy array
      frame_number    : int
      save_csv        : whether to write detections to CSV
      conf_threshold  : YOLO confidence cutoff (default 0.25)
      writer          : csv.writer instance (required if save_csv=True)

    Returns:
      annotated frame (numpy array)
    """
    # 1) Inference with confidence threshold
    result_yolo = model(frame, conf=conf_threshold,iou=.3)[0]

    # 2) Convert to supervision detections
    result_sv = sv.Detections.from_ultralytics(result_yolo)

    # 3) Annotate boxes
    ba = sv.BoxAnnotator()
    img = ba.annotate(frame, result_sv)

    # 4) Overlay frame number
    cv2.putText(
        img,
        text=str(frame_number),
        org=(10, 30),  # (x, y) bottom-left of text
        fontFace=cv2.FONT_HERSHEY_COMPLEX,
        fontScale=1,
        color=(255, 0, 0),  # BGR red
        thickness=2,
        lineType=cv2.LINE_AA
    )

    # 5) Optionally write CSV rows
    if save_csv:
        if writer is None:
            raise ValueError("`writer` must be provided when save_csv=True")
        for idx, (xyxy, conf) in enumerate(zip(result_sv.xyxy, result_sv.confidence), start=1):
            x1, y1, x2, y2 = xyxy
            c_x = (x1 + x2) / 2
            c_y = (y1 + y2) / 2
            # ball_id = idx
            writer.writerow([
                frame_number,
                idx,      # Ball_id
                conf,
                c_x,
                c_y,
                x1, y1, x2, y2
            ])

    return img

In [ ]:
try:
    csv_file = '../Data/detections.csv'
    f = open(csv_file,mode='w',newline='')
    writer = csv.writer(f)
    writer.writerow(['frame_number','ball_id','confidence','c_x','c_y','x1','x2','y1','y2'])
    sv.process_video(source_path=source_file,
                     target_path=target_file,
                     callback=lambda frame,frame_number:run_inference(model, frame,frame_number,save_csv=True,writer=writer))
    logging.info('Inference completed')
finally:
    f.close()
    
